# RB 实验

在随机基准测试中，主要思想是通过对随机序列的一系列量子门操作进行测量，来评估量子门的错误率。这些随机序列的特点是它们会趋向于互相抵消，从而减少系统噪声的影响。通过比较实际测量结果和理论预期，可以估计出量子门的错误率。

实验过程如下：

   * 生成随机序列：生成一系列随机的量子门序列，这些序列涵盖了不同的量子门操作和顺序。

   * 实验操作：在量子设备上依次执行生成的随机序列。
  
   * 测量结果：对每个随机序列，测量量子比特的状态，并记录测量结果。在Clifford群中随机选取$m$个门，构成一个序列；
    计算该序列的等效 $U$ 操作，其逆操作 $\mathrm{U}^{-1}$ 亦属于Clifford群，将 $\mathrm{U}^{-1}$ 添加至序列作为第$m+1$个元素，形成完整序列；
    测量量子比特在该序列作用后处于0态（初始态为0态）的保真度 $y_{mk}$。

   * 分析：通过比较预期测量结果和实际测量结果，可以得出量子门的错误率。通常使用指数递减模型来分析错误率。

   * 错误率估计：通过分析获得的数据，可以估计出不同量子门的平均错误率。
    计算序列长度为m时的平均保真度 $y_m=\frac{1}{K} \sum_1^K y_{m k}$ , 得到序列平均保真度y和序列长度m的关系，并用公式下面拟合：

        $$
        y = A \cdot p^m + B
        $$

        门操作平均错误率 $r_c$ 与拟合参数𝑝的关系：

        $$
        r_c=(1-p) *\left(2^n-1\right) / 2^n
        $$

随着Clifford门数量的增加，门序列的平均成功概率会下降，因为当重复应用包含错误的量子门时，整个门序列的错误概率会单调增加，在RB中，通常假设不同Clifford的噪声超算子相等，在此假设下，门序列的平均成功概率已被证明随Clifford门的数量呈指数衰减。

# 实践

在开始RB实验之前，我们要导入一些包。

In [ ]:
import os
import json
import pathlib
from datetime import datetime
from typing import List, Dict
%matplotlib inline

from originbench.RB.rb_options import *
from originbench.RB.rb import RBExperiment
from originbench.RB.rb_storage import (
    CIRCUITS_ENTRY, RB_RESULTS_FILE, TASKIDS_ENTRY
)
from originbench.util.task_pilot import submit_pilot, query_result_pilot
from originbench.util.task_qcloud import submit_qcloud, query_result_qcloud

导入必要的包后，我们还需要确定RB实验的配置参数，才能正确运行RB实验。

In [ ]:
def save_rb_options(
    options: RBOptions,
    filename: str=RB_OPTIONS_FILE,
) -> None:
    working_dir = options.get_option(WORKING_DIR_ENTRY)
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)


    s = str(pathlib.Path(os.getcwd()) / filename)
    print(f'Saving RB options to {s}')
    options.save_options(filename=filename)

using_qcloud = True
api_key_qcloud = (
    'your api token'
)

dirname = 'qcloud'
experiment_time = datetime.now().strftime('%Y%m%d%H%M')
original_dir = os.getcwd()
working_dir = f'test/RB/{dirname}/{experiment_time}'
working_dir = str(pathlib.Path(os.getcwd()) / working_dir)
print(f'working directory: {working_dir}')

n_qubits = 2
samples = 10
sequence_length = list(range(2, 200, 50))
shots = 1000
api_key = api_key_qcloud

rb_options = RBOptions(
    working_dir=working_dir,
    n_qubits=n_qubits,
    n_samples=samples,
    sequence_length=sequence_length,
    shots=shots,
    api_key=api_key,
    submit_url=None,
    using_qcloud=using_qcloud,
)

save_rb_options(options=rb_options)

## 1. 构建量子线路

In [ ]:
def generate_rb_circuits(options: RBOptions):
    expriment = RBExperiment(options)
    circuits = expriment.build_circuits()

    working_dir = options.get_option(WORKING_DIR_ENTRY)
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)

    with open(RB_RESULTS_FILE, 'w') as f:
        json.dump({CIRCUITS_ENTRY: circuits}, f)


os.chdir(working_dir)

print('generating RB circuits...')
generate_rb_circuits(options=rb_options)
print('RB circuits generated.')

## 2. 提交量子线路
量子线路生成之后，我们就可以将它提交给我们的量子计算机运行。

In [ ]:
def submit_circuits(
    circuits: List[str],
    shots: int,
    api_key: str,
    submit_url: str,
    auto_mapping: bool,
    is_amend: bool,
    using_optimization: bool,
    chip_id: int,
    desc: str='',
    using_qcloud: bool=False,
) -> str:
    if using_qcloud:
        return submit_qcloud(
            circuits=circuits,
            api_key=api_key,
            shots=shots,
            auto_mapping=auto_mapping,
            is_amend=is_amend,
            using_optimization=using_optimization,
            chip_id=chip_id,
        )
    else:
        return submit_pilot(
            circuits=circuits,
            shots=shots,
            api_key=api_key,
            submit_url=submit_url,
            auto_mapping=auto_mapping,
            using_optimization=using_optimization,
            is_amend=is_amend,
            chip_id=chip_id,
            desc=desc,
        )

def submit_rb_circuits(options: RBOptions):

    working_dir = options.get_option(WORKING_DIR_ENTRY)
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)

    with open(RB_RESULTS_FILE, 'r') as f:
        data = json.load(f)
    circuits_lst = data[CIRCUITS_ENTRY]

    # Submit circuits
    using_qcloud = options.get_option(USING_QCLOUD_ENTRY)
    shots = options.get_option(SHOTS_ENTRY)
    api_key = options.get_option(API_KEY_ENTRY)
    submit_url = options.get_option(SUBMIT_URL_ENTRY)
    is_mapping = options.get_option(IS_MAPPING_ENTRY)
    is_amend = options.get_option(IS_AMEND_ENTRY)
    is_optimization = options.get_option(IS_OPTIMIZATION_ENTRY)
    chip_id = options.get_option(CHIP_ID_ENTRY)

    taskid_lst = [
        submit_circuits(
            circuits,
            shots,
            api_key,
            submit_url,
            is_mapping,
            is_amend,
            is_optimization,
            chip_id,
            using_qcloud=using_qcloud
        )
        for circuits in circuits_lst
    ]

    data[TASKIDS_ENTRY] = taskid_lst
    with open(RB_RESULTS_FILE, 'w') as f:
        json.dump(data, f)

print('Submitting RB circuits...')
submit_rb_circuits(options=rb_options)
print('Submission complete.')

## 3. 获取量子线路结果
在获取量子线路结果时，往往会遇到量子线路没有计算完成，我们通常可以等待芯片结果计算完成后一次性获取结果。

In [ ]:
def get_result(
    taskid: str,
    api_key: str,
    submit_url: str,
    using_qcloud: bool,
) -> List[Dict[str, int]]:
    if using_qcloud:
        return query_result_qcloud(taskid, api_key)
    else:
        return query_result_pilot(taskid, submit_url, api_key)

def update_rb_results(options: RBOptions):
    working_dir = options.get_option(WORKING_DIR_ENTRY)
    using_qcloud = options.get_option(USING_QCLOUD_ENTRY)
    submit_url = options.get_option(SUBMIT_URL_ENTRY)
    api_key = options.get_option(API_KEY_ENTRY)
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)

    with open(RB_RESULTS_FILE, 'r') as f:
        data = json.load(f)
    
    taskid_lst = data[TASKIDS_ENTRY]

    for taskid in taskid_lst:
        print(f'Updating result of taskid {taskid}')
        taskid_result = get_result(taskid, api_key, submit_url, using_qcloud)
        with open(f'result_{taskid}.json', 'w') as f:
            json.dump(taskid_result, f)

update_rb_results(options=rb_options)

## 4. 结果处理

当我们获得了所有量子线路的芯片运行结果，我们就可以对这些数据进行分析。

In [ ]:
%matplotlib inline
def process_rb_results(options: RBOptions):
    working_dir = options.get_option(WORKING_DIR_ENTRY)

    if not os.path.exists(working_dir):
        print(f'Working directory {working_dir} does not exist.')
        exit(1)

    with open(RB_RESULTS_FILE, 'r') as f:
        data = json.load(f)
    taskid_lst = data[TASKIDS_ENTRY]

    def get_taskid_result(taskid: str) -> List[Dict[str, int]]:
        with open(f'result_{taskid}.json', 'r') as f:
            data = json.load(f)
        return data

    results = [get_taskid_result(taskid) for taskid in taskid_lst]

    experiment = RBExperiment(options)
    experiment.analyze_results(results)

print('Processing RB results...')
process_rb_results(options=rb_options)
print('Process RB results done.')

In [ ]:
os.chdir(original_dir)